In [1]:
def divider_generator(bin_1_s=None, bin_1_e=None, bin_2_e=None):
    def divider(x):
        if (x >= bin_1_s) & (x <= bin_1_e):
            return 1
        elif (x >= (bin_1_e+1)) & (x <= bin_2_e):
            return 2
        else:
            return 3
    return divider

map_hour = divider_generator(7, 12, 20)

In [13]:
def binner_generator(Bin=None, Range=None, stop=None, expt=None):
    def binner(x):
        Bin = 0.1
        for i in range(1, Range):
            if (x >= stop + Bin*(i - 1)) & (x <= stop + Bin*i):
                return i + 1  
        if x == expt:
            return 1
    return binner

deliver = binner_generator(0.1, 20, 4.1, -5)

In [15]:
deliver(4.1)

2

In [10]:
def base_counter(data, Base, List):
    itemCount = data.groupby([Base], as_index=False)['instance_id'].agg({Base+'_cnt': 'count'})
    data = data.merge(itemCount, on=Base, how='lelf')

    for f in List:
        itemCount = data.groupby([Base, f], on=[Base, f], how='left')
        data[str(f)+Base+'_prob'] = data[str(col)+Base+'_cnt'] / data[Base+'_cnt']

    del data[Base+'_cnt']
    return data

In [20]:
item_list = ['item_id', 'item_brand_id', 'item_city_id', 'item_price_level', 'item_sales_level', 'item_collected_level', 'item_pv_level']

In [21]:
item_list[0]

'item_id'

In [22]:
item_list[1:]

['item_brand_id',
 'item_city_id',
 'item_price_level',
 'item_sales_level',
 'item_collected_level',
 'item_pv_level']

In [28]:
for i in range(len(item_list)-1):
    print(i, i+1)

0 1
1 2
2 3
3 4
4 5
5 6


In [29]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import pandas_profiling as pp
import matplotlib.pyplot as plt
%matplotlib inline

import gossipcat as gc
import Conversion

import warnings
warnings.filterwarnings('ignore')

wd = '/Users/ewenwang/Documents/practice_data/conversion_rate/'

Data = Conversion.Data(wd, data_list=Conversion.new_data, is_all=False)

In [71]:
train, test = Data.train, Data.test

In [72]:
def base_counter(data, Base, List, count_base=True, delete_base=True):
    if count_base:
        itemCount = data.groupby([Base], as_index=False)['instance_id'].agg({Base+'_cnt': 'count'})
        data = data.merge(itemCount, on=Base, how='left')
    for f in List:
        itemCount = data.groupby([Base, f], as_index=False)['instance_id'].agg({str(f)+'_'+Base+'_cnt': 'count'})
        data = data.merge(itemCount, on=[Base, f], how='left')
        data[str(f)+'_'+Base+'_prob'] = data[str(f)+'_'+Base+'_cnt'] / data[Base+'_cnt']
    if delete_base:
        del data[Base+'_cnt']
    return data

user_list = ['user_id', 'user_gender_id', 'user_age_level', 'user_occupation_id', 'user_star_level']
item_list = ['item_id', 'item_brand_id', 'item_city_id', 'item_price_level', 'item_sales_level', 'item_collected_level', 'item_pv_level']
shop_list = ['shop_id', 'shop_review_num_level', 'shop_star_level']

def UserItemShop(data):
    for i in range(len(user_list)-1):   # user
        data = data.pipe(base_counter, Base=user_list[i], List=user_list[i+1:])
    for i in range(len(item_list)-1):   # item
        data = data.pipe(base_counter, Base=item_list[i], List=item_list[i+1:])
    for i in range(len(shop_list)-1):   # shop
        data = data.pipe(base_counter, Base=shop_list[i], List=shop_list[i+1:])
    for i in range(len(user_list)-1):   # user-item
        data = data.pipe(base_counter, Base=user_list[i], List=item_list, delete_base=False)
    for i in range(len(user_list)-1):   # user-shop
        data = data.pipe(base_counter, Base=user_list[i], List=shop_list, count_base=False)
    for i in range(len(shop_list)-1):   # shop-item
        data = data.pipe(base_counter, Base=shop_list[i], List=item_list)
    return data

In [73]:
temp = test

In [74]:
temp = UserItemShop(temp)

In [75]:
temp.head()

,instance_id,item_id,item_brand_id,item_city_id,item_price_level,item_sales_level,item_collected_level,item_pv_level,user_id,user_gender_id,...,item_city_id_shop_review_num_level_cnt,item_city_id_shop_review_num_level_prob,item_price_level_shop_review_num_level_cnt,item_price_level_shop_review_num_level_prob,item_sales_level_shop_review_num_level_cnt,item_sales_level_shop_review_num_level_prob,item_collected_level_shop_review_num_level_cnt,item_collected_level_shop_review_num_level_prob,item_pv_level_shop_review_num_level_cnt,item_pv_level_shop_review_num_level_prob
0,2475218615076601065,2275895163219263378,7838285046767229711,7534238860363577544,7,6,8,14,4379490381248358680,0,...,423,0.341404,672,0.542373,59,0.047619,59,0.047619,117,0.094431
1,398316874173557226,7096238490711246967,8126195666233054089,3948283326616421003,6,9,8,16,2629129219935839101,1,...,164,0.085195,585,0.303896,185,0.096104,104,0.054026,332,0.172468
2,6586402638209028583,7096238490711246967,8126195666233054089,3948283326616421003,6,9,8,16,1092007111346257147,0,...,164,0.085195,585,0.303896,185,0.096104,104,0.054026,332,0.172468
3,1040996105851528465,7096238490711246967,8126195666233054089,3948283326616421003,6,9,8,16,5353289677517820939,1,...,164,0.085195,585,0.303896,185,0.096104,104,0.054026,332,0.172468
4,6316278569655873454,7096238490711246967,8126195666233054089,3948283326616421003,6,9,8,16,1098833840660870025,0,...,164,0.085195,585,0.303896,185,0.096104,104,0.054026,332,0.172468


In [56]:
for i in range(len(item_list)-1):   # item
    test = test.pipe(base_counter, Base=item_list[i], List=item_list[i+1:])

In [58]:
for i in range(len(shop_list)-1):   # shop
    test = test.pipe(base_counter, Base=shop_list[i], List=shop_list[i+1:])

In [60]:
for i in range(len(user_list)-1):   # user-item
    test = test.pipe(base_counter, Base=user_list[i], List=item_list, delete_base=False)

In [67]:
for i in range(len(user_list)-1):   # user-shop
    test = test.pipe(base_counter, Base=user_list[i], List=shop_list, count_base=False)

In [69]:
for i in range(len(shop_list)-1):   # shop-item
    test = test.pipe(base_counter, Base=shop_list[i], List=item_list)

In [85]:
def Combine(data):
    for col in ['user_gender_id', 'user_age_level', 'user_occupation_id', 'user_star_level']:
        data[col] = data[col].apply(lambda x: 0 if x == -1 else x)

    for col in ['item_sales_level', 'item_price_level', 'item_collected_level',
                'user_gender_id', 'user_age_level', 'user_occupation_id', 'user_star_level',
                'shop_review_num_level', 'shop_star_level']:
        data[col] = data[col].astype(str)

    print('item property combinating...')
    data['sale_price'] = data['item_sales_level'] + data['item_price_level']
    data['sale_collect'] = data['item_sales_level'] + data['item_collected_level']
    data['price_collect'] = data['item_price_level'] + data['item_collected_level']

    print('user property combining...')
    data['gender_age'] = data['user_gender_id'] + data['user_age_level']
    data['gender_occ'] = data['user_gender_id'] + data['user_occupation_id']
    data['gender_star'] = data['user_gender_id'] + data['user_star_level']

    print('shop property combining...')
    data['review_star'] = data['shop_review_num_level'] + data['shop_star_level']

    inted_list = ['item_sales_level', 'item_price_level', 'item_collected_level', 
    'user_gender_id', 'user_age_level', 'user_occupation_id', 'user_star_level',
    'shop_review_num_level', 'shop_star_level', 'review_star', 
    'sale_price', 'sale_collect', 'price_collect',
    'gender_age', 'gender_occ', 'gender_star']
    
    for col in inted_list:
        data[col] = data[col].astype(int)

    del data['review_star']

    return data

In [86]:
test = Combine(test)

item property combinating...
user property combining...
shop property combining...


In [87]:
test

,instance_id,item_id,item_brand_id,item_city_id,item_price_level,item_sales_level,item_collected_level,item_pv_level,user_id,user_gender_id,...,hour,day,user_query_day,user_query_day_hour,sale_price,sale_collect,price_collect,gender_age,gender_occ,gender_star
0,2475218615076601065,2275895163219263378,7838285046767229711,7534238860363577544,7,6,8,14,4379490381248358680,0,...,22,25,2,2,67,68,78,1002,2005,3001
1,398316874173557226,7096238490711246967,8126195666233054089,3948283326616421003,6,9,8,16,2629129219935839101,1,...,16,25,2,2,96,98,68,11004,12002,13006
2,6586402638209028583,7096238490711246967,8126195666233054089,3948283326616421003,6,9,8,16,1092007111346257147,0,...,14,25,2,2,96,98,68,1004,2002,3004
3,1040996105851528465,7096238490711246967,8126195666233054089,3948283326616421003,6,9,8,16,5353289677517820939,1,...,15,25,3,1,96,98,68,11003,12002,13007
4,6316278569655873454,7096238490711246967,8126195666233054089,3948283326616421003,6,9,8,16,1098833840660870025,0,...,15,25,1,1,96,98,68,1002,2003,3002
5,868158305045921978,7096238490711246967,8126195666233054089,3948283326616421003,6,9,8,16,158655638471934821,0,...,16,25,2,2,96,98,68,1003,2005,3006
6,5713520501786699854,1171140458228333440,1547983184598012595,8072963182326625214,5,6,6,15,7254617831443442890,0,...,11,25,2,2,65,66,56,1004,2002,3006
7,932945015407923184,7020520887593189887,8324857885829459863,1019055478500227370,7,10,11,14,7588649790698790290,1,...,13,25,1,1,107,1011,711,11001,12005,13002
8,1919197847086752313,8143258757857432437,8324857885829459863,1019055478500227370,8,10,13,15,7745527540524595604,0,...,11,25,1,1,108,1013,813,1002,2002,3007
9,304887065966615346,8143258757857432437,8324857885829459863,1019055478500227370,8,10,13,15,2401138934950272514,0,...,14,25,1,1,108,1013,813,1004,2002,3007
